Task 1: Multi-layer ANN

Hyperparameters:
 1. Number of nodes
 2. Number of layers
 3. Activation function

In [24]:
# Load library
import pandas as pd
import numpy as np
import math
from enum import Enum
from sklearn.model_selection import train_test_split

In [25]:
# Load data
concrete = pd.read_csv('data/concrete_data.csv')
concrete.head()

,cement,blast_furnace_slag,fly_ash,water,superplasticizer,coarse_aggregate,fine_aggregate,age,concrete_compressive_strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [26]:
# Separate X and Y
# Then separate test and train set
# Also do the Cross-Validation (optional)
X = concrete.drop('concrete_compressive_strength', axis = 1)
y = concrete['concrete_compressive_strength']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.7, random_state = 42)

In [27]:
# Activation function
def logistic(x):
    return 1/(1 + math.exp(-x))

def ReLU(x):
    return max(0, x)

def hyperbolic(x):
    return math.tanh(x)

class ActFunc(Enum):
    log = logistic
    relu = ReLU
    hb = hyperbolic

In [28]:
# Neural Network test!
from neuralNet import neuralNet
from layer import layer

network = neuralNet()

network.add(layer(ActFunc.relu,3))
#network.add(layer(ActFunc.relu,8))
#network.add(layer(ActFunc.hb, 10))
network.add(layer(ActFunc.relu,1))

Applying NN on the concrete data

In [40]:
# PSO test!
import pso

swarmsize = 10
alpha = 0.7
beta = 2
gamma = 1.5
delta = 1.5
epsilon = 0.4
n_iter = 10

particle_swarm_opti = pso.PSO(X_train, y_train, network, swarmsize, alpha, beta, gamma, delta, epsilon, n_iter)
opti_particle, best_mae_arr = particle_swarm_opti.optimise()



Current best mae: 46624.956333190195
Current best mae: 42972.463935989246
Current best mae: 42939.82956490358
Current best mae: 30128.83496357125
Current best mae: 26601.338859063893
Current best mae: 20645.5642131235
Current best mae: 15679.900886773376
Current best mae: 11098.107947578694
Current best mae: 8666.053573556203
Current best mae: 6170.799375161087
Current best mae: 4984.456470201047
Current best mae: 3628.6974188931026
Current best mae: 3052.562127940951
Current best mae: 2495.8319819820244
Current best mae: 1835.87338290992
Current best mae: 1638.3404737564706
Current best mae: 1036.3466631701815
Current best mae: 282.8788631779462
Final best mae 282.8788631779462


In [30]:
best_mae_arr

[1981.1088344245757,
 730.8029745410014,
 580.3558388762057,
 166.4686856758687,
 141.0648385341519,
 89.45840522653579,
 34.65535753761032,
 34.47567592264139]

In [31]:
# Testing the optimisation
# Ooooo bad bad result :D
particle_swarm_opti.assessFitness(opti_particle, y_train)

34.47567592264139

In [32]:
# Testing with the test set
weights, bias = particle_swarm_opti.assessFitness_helper(opti_particle) # Extract the optimised weight and bias
# Apply on the NN
y_pred = X_test.apply(network.forwardCalculation, args = (weights, bias), axis = 1)
mse = network.sseCalculation(y_pred, y_test)
mse

35.73954729803863

In [33]:
# Potential ranges of PSO parameters to try
# Could try them like a gridsearch but its probably a bit too much for that, tuning one at a time is probably the way to go

alphaRange = np.arange(0.4,0.9,0.1)
betaRange = np.arange(1.5,2.5,0.1)
gammaRange = np.arange(1.5,2.5,0.1)
deltaRange = np.arange(1.5,2.5,0.1)
epsilonRange = np.arange(0.1,0.5,0.1) # not sure about the range of the learning rate, might need to experiment or look for more sources

In [34]:
#alpha tuning

accuracies = []

for alpha in alphaRange:
    network = neuralNet()

In [35]:
# Try neural network architectures
# 2-10 layers
# 3-10 neurons per layer

layers = range(15,22)

# Try different layers
# Constant 4 neurons per layer, activation func relu

accuraciesForLayers = []
for numLayers in layers:
    network = neuralNet()
    for i in range(numLayers):
        network.add(layer(ActFunc.relu,4))
    layerTestPSO = pso.PSO(X_train, y_train, network, swarmsize, alpha, beta, gamma, delta, epsilon, n_iter,prints=False)
    opti_particle, best_mae_arr = layerTestPSO.optimise()
    accuraciesForLayers.append(best_mae_arr[-1])


In [36]:
accuraciesForLayers

[12.671516033560886,
 36.02132686084142,
 15.427472643768116,
 17.458108364350892,
 22.668423128830145,
 36.02132686084142,
 20.181922898595293]

In [37]:
neurons = range(3,11)

accuraciesForNeuronCounts = []

for neuronCount in neurons:
    network = neuralNet()
    for i in range(5):
        network.add(layer(ActFunc.relu,neuronCount))
    neuronsTestPSO = pso.PSO(X_train, y_train, network, swarmsize, alpha, beta, gamma, delta, epsilon, n_iter,prints=False)
    opti_particle, best_mae_arr = neuronsTestPSO.optimise()
    accuraciesForNeuronCounts.append(best_mae_arr[-1])

In [38]:
accuraciesForNeuronCounts

[18.06420657143566,
 17.914790150586615,
 26.219428354768205,
 36.01887537252918,
 36.02132686084142,
 35.896069046074345,
 36.02132686084142,
 35.61476303081232]

In [41]:
accuraciesForAlphas = []

for a in alphaRange:
    network = neuralNet()
    for i in range(5):
        network.add(layer(ActFunc.relu,5))
    alphaTestPSO = pso.PSO(X_train, y_train, network, swarmsize, a, beta, gamma, delta, epsilon, n_iter,prints=False)
    opti_particle, best_mae_arr = alphaTestPSO.optimise()
    accuraciesForAlphas.append(best_mae_arr[-1])

In [42]:
accuraciesForBetas = []

for b in betaRange:
    network = neuralNet()
    for i in range(5):
        network.add(layer(ActFunc.relu,5))
    betaTestPSO = pso.PSO(X_train, y_train, network, swarmsize, alpha, b, gamma, delta, epsilon, n_iter,prints=False)
    opti_particle, best_mae_arr = betaTestPSO.optimise()
    accuraciesForBetas.append(best_mae_arr[-1])

In [43]:
accuraciesForGammas = []

for g in gammaRange:
    network = neuralNet()
    for i in range(5):
        network.add(layer(ActFunc.relu,5))
    gammaTestPSO = pso.PSO(X_train, y_train, network, swarmsize, alpha, beta, g, delta, epsilon, n_iter,prints=False)
    opti_particle, best_mae_arr = gammaTestPSO.optimise()
    accuraciesForGammas.append(best_mae_arr[-1])

In [44]:
accuraciesForDeltas = []

for d in deltaRange:
    network = neuralNet()
    for i in range(5):
        network.add(layer(ActFunc.relu,5))
    deltaTestPSO = pso.PSO(X_train, y_train, network, swarmsize, alpha, beta, gamma, d, epsilon, n_iter,prints=False)
    opti_particle, best_mae_arr = deltaTestPSO.optimise()
    accuraciesForDeltas.append(best_mae_arr[-1])

In [45]:
accuraciesForEpsilons = []

for e in epsilonRange:
    network = neuralNet()
    for i in range(5):
        network.add(layer(ActFunc.relu,5))
    epsilonTestPSO = pso.PSO(X_train, y_train, network, swarmsize, alpha, beta, gamma, delta, e, n_iter,prints=False)
    opti_particle, best_mae_arr = epsilonTestPSO.optimise()
    accuraciesForEpsilons.append(best_mae_arr[-1])

In [46]:
accuraciesForAlphas

[11187.955357059938,
 4536.702848336024,
 3705.033206378822,
 36.02132686084142,
 36.02132686084142]

In [47]:
accuraciesForBetas

[36.02132686084142,
 36.02132686084142,
 36.02132686084142,
 35.23507257898897,
 36.02132686084142,
 36.02132686084142,
 34.23864760064656,
 36.02132686084142,
 27.136141586224948,
 36.02132686084142]

In [48]:
accuraciesForGammas

[36.02132686084142,
 36.02132686084142,
 36.02132686084142,
 35.811375512024426,
 36.02132686084142,
 556.4635929531196,
 36.02132686084142,
 18.593821785284703,
 36.02132686084142,
 36.02132686084142]

In [50]:
accuraciesForDeltas

[36.02132686084142,
 25.330438571501094,
 36.02132686084142,
 36.02132686084142,
 36.02132686084142,
 13.544943769553884,
 35.6267783222779,
 36.02132686084142,
 36.02132686084142,
 34.92077519213428]

In [51]:
accuraciesForEpsilons

[22375.95185847099, 16543.30489094261, 36.02132686084142, 31.54029105802592]